# Taguchi Array

In [1]:
import numpy as np
import pandas as pd
from tabulate import tabulate
from utils import validate_parameters, score_vals, efficiency_calc

In [2]:
MODULE = 0.8e-3
MC = 0.8467e-3

## Input parameter values
#### Currently assumes we will use values from the paper (added to the dict seperately)

In [7]:
# Designate target parameter values
# in_vals = [z_s, z_r2, x_s, x_r2, Cl, gear_thickness] # Cl_i = clearance (i = 1,2,3)
param_vals = {
    'material' : ['PLA', 'ABS', '--', '--'], #blank values to even out table
    'lubricant' : ['None', 'LI', 'Oil', 'PTFE'],
    'z_sh' : [9, 8, 7][::-1],
    'z_r2' : [83, 82, 80][::-1],
    'x_s' : [-0.5, 0, 1],
    'x_r2' : [-0.5, 0, 0.5],
    'Cl' : [0.4e-3, 0.3e-3, 0.2e-3][::-1], #NOTE - use 0.1 for paper, though they likely use 0.0
    'gear_thickness' : [12, 8, 6][::-1],
}

# Include reference paper values as 1 set
paper_vals = [6, 81, .5, 1, 0.1e-3, 10] # NOTE - 6 = 12 (z_s) / 2.

headers = list(param_vals.keys())
rows = list(zip(*param_vals.values()))

print(tabulate(rows, headers=headers, tablefmt="grid"))

# Put the dictionary into a vector & count total possible designs
quick_param = []
for i in (range(2,len(param_vals))):
    key_i = list(param_vals.keys())[i]
    param_vals[key_i].insert(0, paper_vals[i-2])
    quick_param.append(param_vals[key_i])

design_vals, I1, I2, gr_s, ratios = validate_parameters(quick_param[:-1])

design_vals = np.array(design_vals)

print(f'Design vals shape: {design_vals.shape}') # Note that 8 parameters yield 9 geometric design values

+------------+-------------+--------+--------+-------+--------+--------+------------------+
| material   | lubricant   |   z_sh |   z_r2 |   x_s |   x_r2 |     Cl |   gear_thickness |
+============+=============+========+========+=======+========+========+==================+
| PLA        | None        |      6 |     81 |   0.5 |    1   | 0.0001 |               10 |
+------------+-------------+--------+--------+-------+--------+--------+------------------+
| ABS        | LI          |      7 |     80 |  -0.5 |   -0.5 | 0.0002 |                6 |
+------------+-------------+--------+--------+-------+--------+--------+------------------+
| --         | Oil         |      8 |     82 |   0   |    0   | 0.0003 |                8 |
+------------+-------------+--------+--------+-------+--------+--------+------------------+
| --         | PTFE        |      9 |     83 |   1   |    0.5 | 0.0004 |               12 |
+------------+-------------+--------+--------+-------+--------+--------+--------

## Use L32b Taguchi array to prepare design parameters for each test

In [4]:
Taguchi = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
[1, 1, 2, 2, 2, 2, 2, 2, 2, 2],
[1, 1, 3, 3, 3, 3, 3, 3, 3, 3],
[1, 1, 4, 4, 4, 4, 4, 4, 4, 4],
[1, 2, 1, 1, 2, 2, 3, 3, 4, 4],
[1, 2, 2, 2, 1, 1, 4, 4, 3, 3],
[1, 2, 3, 3, 4, 4, 1, 1, 2, 2],
[1, 2, 4, 4, 3, 3, 2, 2, 1, 1],
[1, 3, 1, 2, 3, 4, 1, 2, 3, 4],
[1, 3, 2, 1, 4, 3, 2, 1, 4, 3],
[1, 3, 3, 4, 1, 2, 3, 4, 1, 2],
[1, 3, 4, 3, 2, 1, 4, 3, 2, 1],
[1, 4, 1, 2, 4, 3, 3, 4, 2, 1],
[1, 4, 2, 1, 3, 4, 4, 3, 1, 2],
[1, 4, 3, 4, 2, 1, 1, 2, 4, 3],
[1, 4, 4, 3, 1, 2, 2, 1, 3, 4],
[2, 1, 1, 4, 1, 4, 2, 3, 2, 3],
[2, 1, 2, 3, 2, 3, 1, 4, 1, 4],
[2, 1, 3, 2, 3, 2, 4, 1, 4, 1],
[2, 1, 4, 1, 4, 1, 3, 2, 3, 2],
[2, 2, 1, 4, 2, 3, 4, 1, 3, 2],
[2, 2, 2, 3, 1, 4, 3, 2, 4, 1],
[2, 2, 3, 2, 4, 1, 2, 3, 1, 4],
[2, 2, 4, 1, 3, 2, 1, 4, 2, 3],
[2, 3, 1, 3, 3, 1, 2, 4, 4, 2],
[2, 3, 2, 4, 4, 2, 1, 3, 3, 1],
[2, 3, 3, 1, 1, 3, 4, 2, 2, 4],
[2, 3, 4, 2, 2, 4, 3, 1, 1, 3],
[2, 4, 1, 3, 4, 2, 4, 2, 1, 3],
[2, 4, 2, 4, 3, 1, 3, 1, 2, 4],
[2, 4, 3, 1, 2, 4, 2, 4, 3, 1],
[2, 4, 4, 2, 1, 3, 1, 3, 4, 2],
])-1

In [5]:
param_cols = ['material','lubricant','z_sh', 'z_r2', 'xs', 'xr2', 'Cl', 'gear thickness']
# need to make a list (columns) of tuples (rows)
test_params = []
# use the Taguchi array to get the parameter values
for i in range(len(Taguchi)):
    Tag_vals = Taguchi[i][:len(param_cols)]
    new_params = tuple()
    for ind, val in enumerate(Tag_vals):
        key_i = list(param_vals.keys())[ind]
        new_params = new_params + (param_vals[key_i][val],)
    test_params.append(new_params)

print(tabulate(test_params, headers=param_cols, tablefmt="grid"))

+------------+-------------+--------+--------+------+-------+--------+------------------+
| material   | lubricant   |   z_sh |   z_r2 |   xs |   xr2 |     Cl |   gear thickness |
+============+=============+========+========+======+=======+========+==================+
| PLA        | None        |      6 |     81 |  0.5 |   1   | 0.0001 |               10 |
+------------+-------------+--------+--------+------+-------+--------+------------------+
| PLA        | None        |      7 |     80 | -0.5 |  -0.5 | 0.0002 |                6 |
+------------+-------------+--------+--------+------+-------+--------+------------------+
| PLA        | None        |      8 |     82 |  0   |   0   | 0.0003 |                8 |
+------------+-------------+--------+--------+------+-------+--------+------------------+
| PLA        | None        |      9 |     83 |  1   |   0.5 | 0.0004 |               12 |
+------------+-------------+--------+--------+------+-------+--------+------------------+
| PLA     

## Calculate design (CAD) parameters from design parameters

In [6]:
# design features:
d_cols = ['z_sh', 'z_p1', 'z_p2', 'z_r2', 'xs', 'xp1', 'xp2', 'xr2', 'r_a [m]']

# need to make a list (columns) of tuples (rows)
d_params = []
for j in range(len(test_params)):
    quick_param = list(test_params[j])[2:-1]
    design_vals, I1, I2, gr_s, ratios = validate_parameters(quick_param)
    design_vals = np.array(design_vals)
    
    new_d = tuple(design_vals)
    # for i in range(len(d_cols)):
    #     new_d = new_d + (design_vals[i])
    d_params.append(new_d)

print(tabulate(d_params, headers=d_cols, tablefmt="grid"))

+--------+--------+--------+--------+------+-------+-----------+-------+-----------+
|   z_sh |   z_p1 |   z_p2 |   z_r2 |   xs |   xp1 |       xp2 |   xr2 |   r_a [m] |
+========+========+========+========+======+=======+===========+=======+===========+
|      6 |     38 |     34 |     81 |  0.5 | 0.125 | 0.0521436 |   1   |    0.0206 |
+--------+--------+--------+--------+------+-------+-----------+-------+-----------+
|      7 |     38 |     29 |     80 | -0.5 | 0     | 0.434038  |  -0.5 |    0.0206 |
+--------+--------+--------+--------+------+-------+-----------+-------+-----------+
|      8 |     36 |     31 |     82 |  0   | 0.125 | 0.107299  |   0   |    0.0212 |
+--------+--------+--------+--------+------+-------+-----------+-------+-----------+
|      9 |     34 |     31 |     83 |  1   | 0     | 0.0443486 |   0.5 |    0.022  |
+--------+--------+--------+--------+------+-------+-----------+-------+-----------+
|      6 |     38 |     31 |     81 | -0.5 | 0.375 | 0.288355  | 